In [70]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

In [71]:
RESULTS_BASE_DIR = "../results/nodeClassificationGraphSummarization5RandomSplits"

In [72]:
df_res = pd.read_csv(os.path.join(RESULTS_BASE_DIR, "results.csv"))
name_map = {
    "NodeClassificationConvolutionMatchingSGCNCoarsener": "ConvMatch",
    "NodeClassificationApproximateConvolutionMatchingSGCNCoarsener": "ApproxConvMatch",
    "NodeClassificationGKSummarizer": "GK"
}

df_res["GraphSummarizer"] = df_res["GraphSummarizer"].replace(name_map)


In [73]:
df_res.keys()

Index(['Graph Summarization Time(s)', 'Training Time(s)', 'Test Evaluation',
       'Validation Evaluation', 'dataset', 'split', 'GraphSummarizer', 'r',
       'p_from', 'depth', 'recoarsen_every', 'model', 'n_layers',
       'hidden_layer_sizes', 'use_layer_norm', 'use_edge_weights',
       'use_input_layer', 'log_softmax', 'softmax_output', 'dropout',
       'learning_loss', 'optimizer', 'lr', 'weight_decay', 'epochs', 'seed',
       'Max GPU memory (B)'],
      dtype='object')

In [74]:
df_res = df_res[df_res['dataset'] == 'Cora']

In [75]:
df_res['Total Time(s)'] = df_res['Training Time(s)'] + df_res['Graph Summarization Time(s)']

In [76]:
df_agg = df_res.groupby(['GraphSummarizer', "r", "model", "dataset", "recoarsen_every", "epochs", "hidden_layer_sizes"], dropna=False).agg(
    mean_test=("Test Evaluation", "mean"),
    std_test=("Test Evaluation", "std"),
    mean_val=("Validation Evaluation", "mean"),
    std_val=("Validation Evaluation", "std"),
    mean_sumtime=('Graph Summarization Time(s)', "mean"),
    std_sumtime=('Graph Summarization Time(s)', "std"),
    mean_traintime=('Training Time(s)', "mean"),
    std_traintime=('Training Time(s)', "std"),
    mean_tottime=('Total Time(s)', "mean"),
    std_tottime=('Total Time(s)', "std")
).reset_index()
for col in ["val", "test", "sumtime", "traintime", "tottime"]:
    df_agg[col] = df_agg[f"mean_{col}"].round(4).astype(str) + " ± " + df_agg[f"std_{col}"].round(4).astype(str)
    df_agg = df_agg.drop(columns=[f"mean_{col}", f"std_{col}"])

In [77]:
df_agg

,GraphSummarizer,r,model,dataset,recoarsen_every,epochs,hidden_layer_sizes,val,test,sumtime,traintime,tottime
0,GK,0.05,GCN,Cora,50,600,256,0.7619 ± 0.0271,0.7315 ± 0.0201,0.0 ± 0.0,36.5654 ± 1.998,36.5654 ± 1.998
1,GK,0.10,GCN,Cora,50,600,256,0.7957 ± 0.0204,0.7647 ± 0.0195,0.0 ± 0.0,33.7705 ± 0.3859,33.7705 ± 0.3859
2,GK,0.25,GCN,Cora,50,600,256,0.8188 ± 0.0194,0.7939 ± 0.019,0.0 ± 0.0,40.3183 ± 1.8721,40.3183 ± 1.8721
